In [50]:
import json
# read api key from tally_api_key.json
TALLY_API_KEY = json.load(open('tally_api_key.json'))['key']

In [51]:
import requests

url = "https://api.tally.xyz/query"

headers = {
  'Api-Key': TALLY_API_KEY,
  'Content-Type': 'application/json'
}

In [52]:
def make_votes_query(chain: str, limit: int, offset: int, votes_limit: int, votes_offset: int) -> str:
    query = """query ProposalsWithVotes {
        proposals(chainId: "%s", pagination: { limit: %d, offset: %d }) {
            id
            votes(pagination: { limit: %d, offset: %d }) {
                id
                voter {
                    id
                }
                support
                reason
                proposal {
                    id
                    governor {
                        id
                    }
                }
                block {
                    timestamp
                }
            }
        }
    }"""
    return query % (chain, limit, offset, votes_limit, votes_offset)

In [53]:
props_count_query = """query ProposalsWithVotes {
  proposals(chainId: "eip155:42161") {
    id
  }
}"""
response = requests.request("POST", url, headers=headers, data=json.dumps({'query': props_count_query}))
data = response.json()
proposals = data['data']['proposals']
len(proposals)

252

In [54]:
import time
import datetime

def get_query_results(limit, offset):
    votes_limit = 3000
    votes_offset = 0
    query = make_votes_query('eip155:42161', limit, offset, votes_limit, votes_offset)
    status_code = 999
    while status_code != 200:
        response = requests.request("POST", url, headers=headers, data=json.dumps({'query': query}))
        status_code = response.status_code
        if status_code != 200:
            print('    error', 'eip155:42161', offset, '-', limit+offset, response.status_code, datetime.datetime.now())
            # wait
            time.sleep(5)
    data = response.json()
    proposals = data['data']['proposals']
    print('    done', 'eip155:42161', offset, '-', limit+offset, response.status_code, datetime.datetime.now())
    if len(proposals) < limit:
        print('    reached limit', 'eip155:42161', offset, response.status_code, datetime.datetime.now())
    return proposals

In [55]:
"""
for all proposals:
    get proposal chunk (limit, offset)
        get votes chunk (limit, offset)
            until limit is reached
        until limit is reached
"""

'\nfor all proposals:\n    get proposal chunk (limit, offset)\n        get votes chunk (limit, offset)\n            until limit is reached\n        until limit is reached\n'

In [56]:
# handling eip155:42161
proposals_42161 = []

print('starting', 'eip155:42161', datetime.datetime.now())

# get proposals in chunks of 50; there are 252 proposals
for offset in range(0, len(proposals), 50):
    result = get_query_results(50, offset)
    proposals_42161.append(result)
    # we've artificially limited the no. of votes fetched to 1000

starting eip155:42161 2023-07-19 12:44:57.883138
    done eip155:42161 0 - 50 200 2023-07-19 12:44:58.621412
    done eip155:42161 50 - 100 200 2023-07-19 12:44:59.181316
    done eip155:42161 100 - 150 200 2023-07-19 12:45:14.240092
    done eip155:42161 150 - 200 200 2023-07-19 12:45:14.984283
    done eip155:42161 200 - 250 200 2023-07-19 12:45:23.890980
    done eip155:42161 250 - 300 200 2023-07-19 12:45:31.953629
    reached limit eip155:42161 250 200 2023-07-19 12:45:31.954060


In [57]:
proposals_42161

[[{'id': '7111974346912460307016744445591582524939682069963197750155992426034337655927',
   'votes': []},
  {'id': '113638556591867963012662177908817688187696698785932907203812977693535248523190',
   'votes': []},
  {'id': '13493017690761946741400093368187230035591034324284495176513333880370190856201',
   'votes': [{'id': 'eip155:42161:0x936139366c5db48543368EE9cD075267d176a02c/13493017690761946741400093368187230035591034324284495176513333880370190856201|eip155:42161:0xA4a3aC7cBA6B584c674737e7a04d760433147287',
     'voter': {'id': 'eip155:42161:0xA4a3aC7cBA6B584c674737e7a04d760433147287'},
     'support': 'FOR',
     'reason': '',
     'proposal': {'id': '13493017690761946741400093368187230035591034324284495176513333880370190856201',
      'governor': {'id': 'eip155:42161:0x936139366c5db48543368EE9cD075267d176a02c'}},
     'block': {'timestamp': '2023-04-10T18:25:51Z'}}]},
  {'id': '103714146047724671567962670629574765893960633329325169312853689645859637618425',
   'votes': [{'id': 'e

In [62]:
big_proposals = []
# print the proposal ids for the proposals with 1000 votes
for i, proposals in enumerate(proposals_42161):
    for proposal in proposals:
        if proposal.get('votes') is not None and len(proposal.get('votes')) == 3000:
            print(i, proposal['id'])
            big_proposals.append(proposal)

5 70545629960586317780628692755032548222173912190231545322320044688071893662480


In [ ]:
def get_large_proposal(offset: int):
    votes_offset = 0
    votes_limit = 1000
    reached_limit = False
    all_votes = []
    while not reached_limit:
        query = make_votes_query('eip155:42161', 1, offset, votes_limit, votes_offset)
        status_code = 999
        while status_code != 200:
            response = requests.request("POST", url, headers=headers, data=json.dumps({'query': query}))
            status_code = response.status_code
            if status_code != 200:
                print('    error', 'eip155:42161 proposal:', offset, ' votes:', votes_offset, '-', votes_limit+votes_offset, response.status_code, datetime.datetime.now())
                # wait
                time.sleep(5)
        data = response.json()
        proposal = data['data']['proposals'][0]
        votes_count = len(proposal['votes'])
        all_votes += proposal['votes']
        print('votes_count', votes_count, 'votes_limit', votes_limit)
        if votes_count < votes_limit:
            print('    reached votes limit', 'eip155:42161 proposal:', offset, ' votes:', votes_offset, '-', votes_limit+votes_offset, response.status_code, datetime.datetime.now())
            reached_limit = True
        else:
            votes_offset += votes_limit
            print('    done', 'eip155:42161 proposal:', offset, ' votes:', votes_offset, '-', votes_limit+votes_offset, response.status_code, datetime.datetime.now())
    proposal['votes'] = all_votes
    # print total no of votes
    print('    total votes', len(all_votes))
    return [proposal]